<a href="https://colab.research.google.com/github/GergelyTuri/sleep/blob/master/notebooks/attractor/Plot_results_attractor_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datasets that has been run through DCA pipeline:
* 3/17/24 setup


In [ ]:
!git clone https://github.com/GergelyTuri/sleep.git

In [ ]:
%cd sleep
!pip install .

In [ ]:
# %cd sleep
# !git pull

In [ ]:
from google.colab import drive
drive.mount("/gdrive")

In [ ]:
import os
from pathlib import Path
import glob
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
from sleep.src import google_drive

In [ ]:
useful_data = google_drive.useful_datasets()
useful_data

In [ ]:
paths_with_dcas = []

# Iterate through each row in the DataFrame
for index, row in useful_data.iterrows():
    # Construct the expected path using values from the current row
    exp_path = google_drive.return_exp_path(row['mouseID'], row['day'], row['sessionID'])

    # Use glob to find all files in the path that end with dcas.png
    dcas_files = glob.glob(os.path.join(exp_path, '*dcas.png'))

    # Check if the list of files is not empty
    if dcas_files:
        # Since we found at least one file that matches, add the exp_path to our list
        # Note: You could also consider adding the full paths of the found files instead
        paths_with_dcas.append(exp_path)


In [ ]:
paths_with_dcas

In [ ]:
def plot_images_grid(paths_with_dcas:list, which_plot: str, savefig: bool=False) -> None:
    """
    Plot a grid of images from the given paths.

    Args:
        paths_with_dcas (list): List of paths containing images.
        which_plot (str): The type of plot to display.
        savefig (bool, optional): Whether to save the plot as a file. Defaults to False.

    Returns:
        None
    """
    # Filter paths to those containing a file ending with 'dca_3d.png'
    images_paths = []
    for path in paths_with_dcas:
        # Find files in the path that end with 'dca_3d.png'
        found_files = glob.glob(os.path.join(path, f"*{which_plot}.png"))
        images_paths.extend(found_files)

    # Determine the grid size (for simplicity, a square grid)
    num_images = len(images_paths)
    grid_size = int(np.ceil(np.sqrt(num_images)))

    # Create a figure to host the grid
    fig, axs = plt.subplots(grid_size, grid_size, figsize=(20, 20))
    axs = axs.flatten()

    for ax, img_path in zip(axs, images_paths):
        full_path = Path(img_path)
        path_parts = list(full_path.parts)
        title = "\n".join(path_parts[-4:])
        # Open and display the image
        img = Image.open(img_path)
        ax.imshow(img)
        ax.axis("off")  # Hide axes
        ax.set_title(title, fontsize=10)

    # Hide any unused subplots
    for ax in axs[len(images_paths) :]:
        ax.axis("off")

    plt.tight_layout()
    if savefig:
        plt.savefig(f"{which_plot}_plots.png", dpi=300)
    plt.close()

FYI: the plots will be saved locally in the folder where the notebook is running. Don't forget to download them if you want to keep them otherwise they will disappear after the notebook was shut down.

In [ ]:
plots_to_print = [
    "composite",
    "interval_plot",
    "dcas",
    "dca_scatter",
    "dca_3d",]
# Assuming paths_with_dcas is already populated
for plot in plots_to_print:
    plot_images_grid(paths_with_dcas, which_plot=plot, savefig=True)